### Link of the Pycaret docs
#### https://pycaret.readthedocs.io/en/latest/api/time_series.html

In [1]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment
import pmdarima as pm


In [2]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_br.xlsx', sheet_name='Sheet1')
print(df.head(10))

        date   value      liter
0 2010-12-31  238.44        NaN
1 2011-12-31  248.58        NaN
2 2012-12-31  249.03        NaN
3 2013-12-31  257.58        NaN
4 2014-12-31  260.80    70939.0
5 2015-12-31  258.90   141617.0
6 2016-12-31  257.39   200740.0
7 2017-12-31  255.06   412770.0
8 2018-12-31  259.33   720236.0
9 2019-12-31  266.04  1330163.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    14 non-null     datetime64[ns]
 1   value   14 non-null     float64       
 2   liter   10 non-null     float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 464.0 bytes


In [4]:
df.drop(['liter'], axis = 1,inplace=True)

In [5]:
df.set_index('date',inplace=True)

In [6]:
df=df.sort_index(axis=0)
print(df.head(10))

             value
date              
2010-12-31  238.44
2011-12-31  248.58
2012-12-31  249.03
2013-12-31  257.58
2014-12-31  260.80
2015-12-31  258.90
2016-12-31  257.39
2017-12-31  255.06
2018-12-31  259.33
2019-12-31  266.04


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. 

When integer is passed it means N continuous points in the future without any gap. 

If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [7]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(14, 1)"
5,Transformed data shape,"(14, 1)"
6,Transformed train set shape,"(13, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [8]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

## Checking Train-Test Split

In [9]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

## Comparing Models

The most basic assumption of time series forecasting is that past patterns continue into the future. 
Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, 
the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

In [10]:
top4 = exp.compare_models(n_select = 4)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:06:21
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,Naive Forecaster


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
dt_cds_dt,Decision Tree w/ Cond. Deseasonalize & Detrending,2.0424,1.6007,7.9633,7.9633,0.0278,0.0286,0.5967
ada_cds_dt,AdaBoost w/ Cond. Deseasonalize & Detrending,2.0424,1.6007,7.9633,7.9633,0.0278,0.0286,0.1967
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.0426,1.6008,7.9637,7.9637,0.0278,0.0286,0.5300
gbr_cds_dt,Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.0472,1.6044,7.9827,7.9827,0.0279,0.0286,0.1800
exp_smooth,Exponential Smoothing,2.2320,1.7477,8.6761,8.6761,0.0303,0.0312,0.0600
polytrend,Polynomial Trend Forecaster,2.2320,1.7478,8.6762,8.6762,0.0303,0.0312,4.0500
ets,ETS,2.2321,1.7478,8.6763,8.6763,0.0303,0.0312,0.0900
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,2.2401,1.7546,8.7305,8.7305,0.0306,0.0314,0.2333
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,2.2402,1.7545,8.7215,8.7215,0.0305,0.0314,0.1833
knn_cds_dt,K Neighbors w/ Cond. Deseasonalize & Detrending,2.2528,1.7647,8.7676,8.7676,0.0306,0.0315,0.9067


## Lag and Difference
Time series data modeling is to find patterns in the process of normalizing nonstationary time series data. 
Lag and difference are the most representative methods used in the process of making a stationary time series into a stationary time series.

## Difference Plot Using Order

In [ ]:
exp.plot_model(plot="diff", 
               data_kwargs={"order_list": [1, 2], "acf": True, "pacf": True},
               fig_kwargs = {"height":700,"width":1000,"template": "simple_white","hoverinfo":"skip","big_data_threshold":100})

## xgboost_cds_dt


In [11]:
xgboost_cds_dt = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = xgboost_cds_dt, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.2524,0.2035,1.0958,1.0958,0.0041,0.0041
1,2020,0.8436,0.6622,3.3914,3.3914,0.0128,0.0127
2,2021,5.0317,3.9367,19.4040,19.4040,0.0666,0.0689
Mean,NaT,2.0426,1.6008,7.9637,7.9637,0.0278,0.0286
SD,NaT,2.1274,1.6623,8.1436,8.1436,0.0276,0.0287


Processing:   0%|          | 0/4 [00:00<?, ?it/s]


## ada_cds_dt


In [12]:
ada = exp.create_model('ada_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = ada, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.2524,0.2035,1.0958,1.0958,0.0041,0.0041
1,2020,0.8433,0.6619,3.3901,3.3901,0.0128,0.0127
2,2021,5.0317,3.9366,19.4040,19.4040,0.0666,0.0689
Mean,NaT,2.0424,1.6007,7.9633,7.9633,0.0278,0.0286
SD,NaT,2.1274,1.6624,8.1438,8.1438,0.0276,0.0287


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

## dt_cds_dt

In [13]:
dt = exp.create_model('dt_cds_dt')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = dt, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

In [14]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = theta, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.predict_model(theta, fh = 8)


,y_pred
2023,294.0960
2024,295.4441
2025,296.7921
2026,298.1401
2027,299.4881
2028,300.8362
2029,302.1842
2030,303.5322


## ETS

In [15]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = ets, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.predict_model(ets, fh = 8)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.0770,0.0621,0.3345,0.3345,0.0013,0.0013
1,2020,1.0290,0.8077,4.1368,4.1368,0.0156,0.0155
2,2021,5.5902,4.3736,21.5577,21.5577,0.0740,0.0768
Mean,NaT,2.2321,1.7478,8.6763,8.6763,0.0303,0.0312
SD,NaT,2.4061,1.8815,9.2398,9.2398,0.0315,0.0328


,y_pred
2023,278.4608
2024,281.1569
2025,283.8529
2026,286.5490
2027,289.2450
2028,291.9411
2029,294.6371
2030,297.3332


## ARIMA

In [17]:
auto_arima = exp.create_model('auto_arima')
exp.plot_model(plot = 'decomp', 
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = auto_arima, 
               plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.predict_model(auto_arima, fh = 8)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:10:02
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Auto ARIMA


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,0.9325,0.7518,4.0491,4.0491,0.0152,0.0153
1,2020,0.1481,0.1163,0.5954,0.5954,0.0022,0.0022
2,2021,7.2223,5.6506,27.8519,27.8519,0.0956,0.1004
Mean,NaT,2.7676,2.1729,10.8322,10.8322,0.0377,0.0393
SD,NaT,3.1662,2.4727,12.1171,12.1171,0.0413,0.0435


,y_pred
2023,295.8133
2024,300.2267
2025,304.6400
2026,309.0533
2027,313.4667
2028,317.8800
2029,322.2933
2030,326.7067


# LITER

In [17]:
#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_br.xlsx', sheet_name='Sheet1')
print(df.head(10))

        date   value      liter
0 2010-12-31  238.44        NaN
1 2011-12-31  248.58        NaN
2 2012-12-31  249.03        NaN
3 2013-12-31  257.58        NaN
4 2014-12-31  260.80    70939.0
5 2015-12-31  258.90   141617.0
6 2016-12-31  257.39   200740.0
7 2017-12-31  255.06   412770.0
8 2018-12-31  259.33   720236.0
9 2019-12-31  266.04  1330163.0


In [18]:
df= df.dropna()

In [19]:
df.drop(['value'], axis = 1,inplace=True)

In [20]:
df.set_index('date',inplace=True)

In [21]:
df=df.sort_index(axis=0)
print(df.head(10))

                liter
date                 
2014-12-31    70939.0
2015-12-31   141617.0
2016-12-31   200740.0
2017-12-31   412770.0
2018-12-31   720236.0
2019-12-31  1330163.0
2020-12-31  2430153.0
2021-12-31  3296386.0
2022-12-31  4141401.0
2023-12-31  4419242.0


In [22]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=15, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,15
1,Target,liter
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(10, 1)"
5,Transformed data shape,"(10, 1)"
6,Transformed train set shape,"(9, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [23]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

In [24]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

In [26]:
top4 = exp.compare_models(n_select = 4)

## ets

In [27]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.predict_model(ets, fh = 8)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:21:19
Status,. . . . . . . . . . . . . . . . . .,Selecting Estimator
Estimator,. . . . . . . . . . . . . . . . . .,ETS


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.9462,1.5199,490144.1910,490144.1910,0.2017,0.2243
1,2020,0.5942,0.4351,233640.2091,233640.2091,0.0709,0.0685
2,2021,0.0463,0.0358,21337.1041,21337.1041,0.0052,0.0051
Mean,NaT,0.8622,0.6636,248373.8347,248373.8347,0.0926,0.0993
SD,NaT,0.7985,0.6270,191673.0392,191673.0392,0.0817,0.0921


,y_pred
2023,4.986399e+06
2024,5.831395e+06
2025,6.676391e+06
2026,7.521387e+06
2027,8.366383e+06
2028,9.211379e+06
2029,1.005637e+07
2030,1.090137e+07


## POLYTREND

In [ ]:
polytrend = exp.create_model('polytrend')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

## exp_smooth


In [ ]:
exp_smooth = exp.create_model('exp_smooth')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":100})

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:26:16
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Exponential Smoothing


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.9620,1.5322,494114.0214,494114.0214,0.2033,0.2263
1,2020,0.5795,0.4244,227873.4578,227873.4578,0.0691,0.0668
2,2021,0.0563,0.0436,25953.0827,25953.0827,0.0063,0.0062
Mean,NaT,0.8659,0.6667,249313.5206,249313.5206,0.0929,0.0998
SD,NaT,0.8039,0.6314,191726.2342,191726.2342,0.0822,0.0928


## THETA

In [ ]:
theta = exp.create_model('theta')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.predict_model(theta, fh = 8)


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,3.4422,2.6882,866912.4743,866912.4743,0.3567,0.4342
1,2020,1.3088,0.9584,514630.7053,514630.7053,0.1561,0.1693
2,2021,0.8664,0.6707,399239.1080,399239.1080,0.0964,0.1013
Mean,NaT,1.8725,1.4391,593594.0959,593594.0959,0.2031,0.2349
SD,NaT,1.1246,0.8910,198923.7830,198923.7830,0.1113,0.1436


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

,y_pred
2023,4.657164e+06
2024,4.916517e+06
2025,5.175870e+06
2026,5.435223e+06
2027,5.694576e+06
2028,5.953929e+06
2029,6.213282e+06
2030,6.472635e+06


## XGBOOST

In [ ]:
xgboost = exp.create_model('xgboost_cds_dt')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})



In [ ]:
ets = exp.create_model('ets')
exp.plot_model(plot = 'decomp',
               data_kwargs = {'type' : 'multiplicative'},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = ets, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,2019,1.9609,1.8623,899460.9465,899460.9465,0.3701,0.4542
1,2020,2.0020,1.8009,1346269.2401,1346269.2401,0.4084,0.5132
2,2021,0.0296,0.0274,21337.1020,21337.1020,0.0052,0.0051
Mean,NaT,1.3308,1.2302,755689.0962,755689.0962,0.2612,0.3242
SD,NaT,0.9203,0.8509,550372.0265,550372.0265,0.1817,0.2269


In [ ]:
exp.predict_model(ets, fh = 8)

,y_pred
2023,4.986399e+06
2024,5.831395e+06
2025,6.676391e+06
2026,7.521387e+06
2027,8.366383e+06
2028,9.211379e+06
2029,1.005637e+07
2030,1.090137e+07


## ALL GRAPHS

In [ ]:
exp.plot_model(estimator = light, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = polytrend, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = exp_smooth, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = theta, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})
exp.plot_model(estimator = xgboost, plot = 'forecast', 
               data_kwargs = {'fh' : 8},
               fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})